...
#1 Import Data and RDD operations
(a) Create an RDD by loading the txt file from:. "/labdata/shakespear.txt"
(b) Create an RDD1 with each element being a single word from the whole text
(c) Create an RDD2 containing key-value pairs of the form: (line_number, line_text)
(d) Create an RDD3 containing key-value pairs of the form: (line_number, Array_of_line_words)
(e) Create an RDD4 containing key-value pairs of the form: (line_number, (word, frequency))
(f) Create an RDD5 containing key-value pairs of the form: (word, (line_number, frequency))
(g) Create an RDD6 containing key-value pairs of the form: (word, list_of_(line_number, frequency)). That is, for each word, gather all line-frequency pairs in a list.
(h) Create an RDD7 containing key-value pairs of the form: (word, list_of_lines). That is, for each word, gather all lines in a list.
(i) Create an RDD8 containing key-value pairs of the form: (word, total_frequency_of_word_in_text).
(j) Create an RDD9 containing key-value pairs of the form: ((word,line), frequency). That is, the key is the combination of word and line.
(k) Create an RDD10 which counts the above word-line combinations.
(l) Create an RDD11 containing key-value pairs of the form: (word, (line, total_frequency_of_word_in_line))
(m) Create an RDD12 that aggregates the contents of RRD11 into records of the form: (word, (number_of_lines, total_frequency_of_word_in_text))

#2. Implement data 
(a) Create an RDD by loading the /labdata/nyctaxisub.txt (1st line is header)
(b) Create a Dataframe from the above RDD by Inferring the schema using reflection or programmatically.
(c) Find the medallions with the most passengers between 2013-02-09 and 2013-02-11.
(d) Count all the medallion between the dates in (c)
(e) Find the medallions that lasted over 900sec and order them by passenger count in descending order. Date and delivery place
...

In [ ]:
#1
(a)
RDD=sc.textFile("/labdata/sheakspear.txt")
RDD.take(10)

(b)
RDD1=RDD.flatMap(lambda x: x.split(" "))
RDD1.take(10)

(c) 
RDD2=RDD.zipWithIndex().map(lambda x: (x[1] ,x[0]))
RDD2.take(10)

(d) 
RDD3=RDD.zipWithIndex().map(lambda x: (x[1] ,x[0].split(" ")))
RDD3.take(10)

(e)
rdd49=RDD.zipWithIndex().flatMap(lambda x: ((x[1],(i,1)) for i in x[0].split(" ")))
rdd4=rdd49.map(lambda x: ((x[0], x[1][0]), x[1][1])).reduceByKey(lambda x,y: x+y)

RDD4=rdd4.map(lambda x: (x[0][0], (x[0][1], x[1]))).reduceByKey(lambda x, y: x+y)
RDD4.take(10)

(f) 
RDD5=rdd4.map(lambda x: (x[0][1], (x[0][0], x[1]))).reduceByKey(lambda x,y:(x+y))
RDD5.take(10)

(g) 
RDD6=rdd4.map(lambda x: (x[0][1], [(x[0][0], x[1])])).reduceByKey(lambda x,y: x+y)
RDD6.take(10)

(h) 
rdd7=RDD.zipWithIndex().flatMap(lambda x: ((i,x[1]) for i in x[0].split(" ")))
rdd7=rdd7.map(lambda x: (x[0], [x[1]]))
RDD7=rdd7.reduceByKey(lambda x,y: x+y)
RDD7.take(10)

(i)
RDD8=RDD1.map(lambda x:(x,1)).reduceByKey(lambda x,y:(x+y))
RDD8.take(10)

(j)
RDD9=rdd49.map(lambda x: ((x[1][0], x[0]), x[1][1])).reduceByKey(lambda x,y: x+y)
RDD9.take(10)

(k)
RDD10=RDD9.count()
RDD10

(l)
RDD11=rdd49.map(lambda x: (x[1][0], (x[0], x[1][1]))).reduceByKey(lambda x,y: x+y)
RDD11.take(10)

(m)
RDD12=RDD7.join(RDD8)
RDD12.take(10)


#2
(a)
RDDa=sc.textFile("/labdata/nyctaxisub.txt")
RDDa.take(10)

(b)
df = sqlContext.read.load("/labdata/nyctaxisub.txt", format='com.databricks.spark.csv', header='true', inferSchema='true')
df.dtypes
df.show()

(c)
from pyspark.sql import functions as F
c=df.filter(F.col("dropoff_datetime").between('2013-02-09 00:00:00','2013-02-11 23:59:59'))
C=c.orderBy(F.col("passenger_count").desc()).select("medallion").show()

(d)
D=c.orderBy(F.col("passenger_count").desc()).select("medallion").count()
D

(e)
e=df.filter(F.col("trip_time_in_secs")>(F.lit("900")))
E=e.orderBy(F.col("passenger_count").desc()).select("medallion").show()
